In [1]:

import numpy as np
import pandas as pd
import rdkit as rk
from rdkit import Chem 
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [4]:
des_list = [x[0] for x in Descriptors._descList]
len(des_list)

210

In [5]:
df = pd.read_csv('aromatase_inhibitors.csv')
df.head()

,Molecule ChEMBL ID,Smiles,pChEMBL Value
0,CHEMBL1170678,COc1cc(/C=C(\Cn2ccnc2)c2ccc([N+](=O)[O-])cc2)c...,7.16
1,CHEMBL308537,CO[C@@H]1CC2C3CCC(=O)C3(C)CCC2C2(C)CCCC=C12,9.89
2,CHEMBL1083353,O=c1c2ccccc2sc2c(Cn3ccnc3)cccc12,8.40
3,CHEMBL454705,CC1=C[C@@H]2c3c(O)cc(-c4cc5ccc(O)cc5o4)cc3O[C@...,5.12
4,CHEMBL457679,CC(C)=CCc1cc(-c2oc3cc(O)cc(O)c3c(=O)c2O)ccc1O,7.00


In [6]:
df['Molecule'] = df['Smiles'].apply(Chem.MolFromSmiles)

df.head()

,Molecule ChEMBL ID,Smiles,pChEMBL Value,Molecule
0,CHEMBL1170678,COc1cc(/C=C(\Cn2ccnc2)c2ccc([N+](=O)[O-])cc2)c...,7.16,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...
1,CHEMBL308537,CO[C@@H]1CC2C3CCC(=O)C3(C)CCC2C2(C)CCCC=C12,9.89,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...
2,CHEMBL1083353,O=c1c2ccccc2sc2c(Cn3ccnc3)cccc12,8.40,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...
3,CHEMBL454705,CC1=C[C@@H]2c3c(O)cc(-c4cc5ccc(O)cc5o4)cc3O[C@...,5.12,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...
4,CHEMBL457679,CC(C)=CCc1cc(-c2oc3cc(O)cc(O)c3c(=O)c2O)ccc1O,7.00,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...


In [8]:
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(des_list)
descriptors = [calculator.CalcDescriptors(mol) for mol in df['Molecule']]

# Create a new DataFrame with descriptors
descriptors_df = pd.DataFrame(descriptors, columns=des_list)

# Concatenate the original DataFrame with the descriptors DataFrame
df_with_descriptors = pd.concat([df, descriptors_df], axis=1)

# Display the resulting DataFrame
df_with_descriptors.head()

,Molecule ChEMBL ID,Smiles,pChEMBL Value,Molecule,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CHEMBL1170678,COc1cc(/C=C(\Cn2ccnc2)c2ccc([N+](=O)[O-])cc2)c...,7.16,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...,10.921258,10.921258,0.059056,-0.406788,0.358459,11.259259,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL308537,CO[C@@H]1CC2C3CCC(=O)C3(C)CCC2C2(C)CCCC=C12,9.89,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...,12.475786,12.475786,0.022121,-0.022121,0.665030,50.863636,...,0,0,0,0,0,0,0,0,0,0
2,CHEMBL1083353,O=c1c2ccccc2sc2c(Cn3ccnc3)cccc12,8.40,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...,12.630143,12.630143,0.117436,0.117436,0.528456,11.238095,...,0,0,0,0,0,0,0,0,0,0
3,CHEMBL454705,CC1=C[C@@H]2c3c(O)cc(-c4cc5ccc(O)cc5o4)cc3O[C@...,5.12,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...,11.535819,11.535819,0.028273,-1.566627,0.143976,23.642857,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL457679,CC(C)=CCc1cc(-c2oc3cc(O)cc(O)c3c(=O)c2O)ccc1O,7.00,<rdkit.Chem.rdchem.Mol object at 0x0000025A7AF...,12.410893,12.410893,0.056275,-0.807710,0.532609,10.846154,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X = df_with_descriptors.drop(['Molecule ChEMBL ID','Smiles', 'Molecule' ], axis=1)
y = df['pChEMBL Value']

In [13]:
X = X.drop('pChEMBL Value', axis=1)

In [14]:
def remove_correlated_features(descriptors):
    # Calculate correlation
    correlated_matrix = descriptors.corr().abs()

    # Upper triangle of correlation matrix
    upper_triangle = correlated_matrix.where(np.triu(np.ones(correlated_matrix.shape),k=1).astype(np.bool))

    # Identify columns that have above 0.9 values of correlation
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] >= 0.9)]
    print(to_drop)
    descriptors_correlated_dropped = descriptors.drop(columns=to_drop, axis=1)
    return descriptors_correlated_dropped   

In [15]:
descriptors_new = remove_correlated_features(X)
descriptors_new

C:\Users\HP\AppData\Local\Temp\ipykernel_74560\2887473889.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_triangle = correlated_matrix.where(np.triu(np.ones(correlated_matrix.shape),k=1).astype(np.bool))


['MaxEStateIndex', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'SMR_VSA4', 'SlogP_VSA5', 'SlogP_VSA6', 'VSA_EState10', 'VSA_EState6', 'FractionCSP3', 'HeavyAtomCount', 'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticRings', 'NumAromaticCarbocycles', 'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 'NumSaturatedRings', 'MolMR', 'fr_Al_OH_noTert', 'fr_Ar_N', 'fr_COO2', 'fr_C_O_noCOO', 'fr_Nhpyrrole', 'fr_benzene', 'fr_imide', 'fr_ketone_Topliss', 'fr_nitrile', 'fr_nitro_arom', 'fr_nitro_arom_nonortho', 'fr_phenol', 'fr_phenol_noOrthoHbond']


,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,FpDensityMorgan1,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,10.921258,0.059056,-0.406788,0.358459,11.259259,365.389,0,0.268965,-0.496602,1.148148,...,0,0,0,0,0,0,0,0,0,0
1,12.475786,0.022121,-0.022121,0.665030,50.863636,302.458,0,0.138555,-0.377079,1.136364,...,0,0,0,0,0,0,0,0,0,0
2,12.630143,0.117436,0.117436,0.528456,11.238095,292.363,0,0.195377,-0.333018,1.047619,...,0,0,0,0,0,0,0,0,0,0
3,11.535819,0.028273,-1.566627,0.143976,23.642857,562.574,0,0.285340,-0.507823,0.738095,...,0,0,0,0,0,0,0,0,0,0
4,12.410893,0.056275,-0.807710,0.532609,10.846154,354.358,0,0.238289,-0.507678,1.038462,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290,10.628424,0.009495,-1.197538,0.577737,50.041667,336.472,0,0.249529,-0.384351,1.166667,...,0,0,0,0,0,0,0,0,0,0
2291,6.060979,0.219295,-0.219295,0.555212,12.416667,339.782,0,0.137295,-0.496741,1.125000,...,0,0,0,0,0,0,0,0,0,0
2292,12.071668,0.053004,-0.053004,0.700686,10.578947,252.269,0,0.192821,-0.496624,1.000000,...,0,0,0,0,0,0,0,0,0,0
2293,11.942522,0.079502,-0.115645,0.707998,10.666667,238.242,0,0.192821,-0.507822,0.944444,...,0,0,0,0,0,0,0,0,0,0


In [16]:
from sklearn.feature_selection import VarianceThreshold

def remove_low_variance(input_data, threshold=0.1):
    selection = VarianceThreshold(threshold)
    selection.fit(input_data)
    return input_data[input_data.columns[selection.get_support(indices=True)]]

X = remove_low_variance(descriptors_new, threshold=0.1)
X

,MaxAbsEStateIndex,MinEStateIndex,SPS,MolWt,BCUT2D_MWHI,BCUT2D_MRHI,BCUT2D_MRLOW,AvgIpc,BalabanJ,HallKierAlpha,...,fr_bicyclic,fr_ether,fr_halogen,fr_imidazole,fr_ketone,fr_methoxy,fr_para_hydroxylation,fr_pyridine,fr_sulfonamd,fr_unbrch_alkane
0,10.921258,-0.406788,11.259259,365.389,16.628323,5.819353,-0.384441,3.200039,2.034026,-3.61,...,0,2,0,1,0,2,0,0,0,0
1,12.475786,-0.022121,50.863636,302.458,16.474998,5.867281,-0.134271,2.815371,1.738451,-0.63,...,5,1,0,0,1,1,0,0,0,0
2,12.630143,0.117436,11.238095,292.363,32.133500,7.242798,0.803351,3.015370,2.022576,-2.46,...,2,0,0,1,0,0,0,0,0,0
3,11.535819,-1.566627,23.642857,562.574,16.706093,5.843183,-0.197951,3.352059,1.383230,-5.24,...,5,2,0,0,0,0,0,0,0,0
4,12.410893,-0.807710,10.846154,354.358,16.359248,5.877240,0.437949,2.563786,2.226855,-3.41,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290,10.628424,-1.197538,50.041667,336.472,17.253287,5.364780,-0.555849,2.445503,1.863973,-0.42,...,3,1,0,0,0,0,0,0,0,0
2291,6.060979,-0.219295,12.416667,339.782,35.495692,6.300923,0.414831,3.256642,1.853537,-2.79,...,1,1,1,0,0,1,0,0,0,0
2292,12.071668,-0.053004,10.578947,252.269,16.465912,5.795549,0.414160,2.471295,2.257927,-2.55,...,1,1,0,0,0,1,0,0,0,0
2293,11.942522,-0.115645,10.666667,238.242,16.339080,5.794319,0.474029,2.388256,2.301484,-2.55,...,1,0,0,0,0,0,0,0,0,0


In [17]:

from sklearn.metrics import matthews_corrcoef, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, RandomizedSearchCV



In [18]:


# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each model
param_grids = {
    'LinearRegression': {},
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 30],
        'min_samples_split': [2, 5, 10],
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids = {}
for model_name, model in models.items():
    #print(f'Training and tuning {model_name}...')
    grids[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grids[model_name].fit(X_train, y_train)
    best_params = grids[model_name].best_params_
    best_score = np.sqrt(-1 * grids[model_name].best_score_)
    
    
    y_train_pred = grids[model_name].predict(X_train)
    y_test_pred = grids[model_name].predict(X_test)
    r_score = r2_score(y_train, y_train_pred)
    rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

    
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')
    print(f'Best Rscore for {model_name}: {r_score}\n')
    print(f'Best RMSE for {model_name}: {rmse}\n')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for LinearRegression: {}
Best RMSE for LinearRegression: 0.9888164620244458

Best Rscore for LinearRegression: 0.4655877096485871

Best RMSE for LinearRegression: 0.9184434901687781

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for RandomForest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}
Best RMSE for RandomForest: 0.771741670033527

Best Rscore for RandomForest: 0.9024587525126135

Best RMSE for RandomForest: 0.39238136563861026

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200}
Best RMSE for XGBoost: 0.7783598823429576

Best Rscore for XGBoost: 0.9256781410977558

Best RMSE for XGBoost: 0.34250940654953205



In [26]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6)
model.fit(X_train, y_train) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
pred = model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, pred)))

0.8533478386188055


In [31]:
model.score()

0.9256781410977558